In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from pprint import pprint
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats

2023-12-01 10:51:51.409009: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 10:51:51.452245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 10:51:51.452278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 10:51:51.453416: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 10:51:51.459606: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 10:51:51.460349: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions

FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions


PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions

In [3]:
hand_model_path = './models/hand_landmarkr_full.task'
face_model_path = './models/face_landmarker.task'
pose_model_path = './models/pose_landmarker_heavy.task'

In [4]:
handOptions = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=hand_model_path),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=2)

faceOptions = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=face_model_path),
    running_mode=VisionRunningMode.VIDEO)

poseOptions = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=pose_model_path),
    running_mode=VisionRunningMode.VIDEO)

In [5]:
def initModels():
    handLandmarker=HandLandmarker.create_from_options(handOptions)
    faceLandmarker=FaceLandmarker.create_from_options(faceOptions)
    poseLandmarker=PoseLandmarker.create_from_options(poseOptions)
    return handLandmarker,faceLandmarker,poseLandmarker
handLandmarker=HandLandmarker.create_from_options(handOptions)
faceLandmarker=FaceLandmarker.create_from_options(faceOptions)
poseLandmarker=PoseLandmarker.create_from_options(poseOptions)
# initModels()

I0000 00:00:1701408126.973608  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701408126.977047  108779 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1701408127.029178  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701408127.030356  108798 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
W0000 00:00:1701408127.030657  108640 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
I0000 00:00:1701408127.060629  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701408127.061806  108817 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer:

In [6]:
def draw_landmarks_on_image(rgb_image, pose_detection_result,hand_detection_result):
  pose_landmarks_list = pose_detection_result.pose_landmarks
  
  hand_landmarks_list = hand_detection_result.hand_landmarks
  handedness_list = hand_detection_result.handedness
  
  annotated_image = np.copy(rgb_image)

  MARGIN = 10  # pixels
  FONT_SIZE = 1
  FONT_THICKNESS = 1
  HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
        annotated_image,
        pose_landmarks_proto,
        solutions.pose.POSE_CONNECTIONS,
        solutions.drawing_styles.get_default_pose_landmarks_style())
    
  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
        annotated_image,
        hand_landmarks_proto,
        solutions.hands.HAND_CONNECTIONS,
        solutions.drawing_styles.get_default_hand_landmarks_style(),
        solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)
  return annotated_image

In [7]:
def save_data(pose_landmarket_result, hand_landmarker_result, name=None,npReturn=False):
    
    if len(pose_landmarket_result.pose_landmarks) > 0:
        finalPoseData: np.ndarray = np.array([[i.x, i.y, i.z]
                                              for i in pose_landmarket_result.pose_landmarks[0]]).flatten()
    else:
        finalPoseData = np.zeros((99,))
    finalLeftHandData = np.zeros((63,))
    finalRightHandData = np.zeros((63,))
    for idx in range(len(hand_landmarker_result.hand_landmarks)):
        
        if hand_landmarker_result.handedness[idx][0].category_name == "Left":
            finalLeftHandData=np.array([[i.x, i.y, i.z] for i in hand_landmarker_result.hand_landmarks[[idx][0]]]).flatten()
        else:
            finalRightHandData = np.array([[i.x, i.y, i.z] for i in hand_landmarker_result.hand_landmarks[[idx][0]]]).flatten()

    if npReturn:
        return np.concatenate([ finalPoseData, finalLeftHandData, finalRightHandData])
    np.save(name, np.concatenate([ finalPoseData, finalLeftHandData, finalRightHandData]))

In [8]:
cap = cv2.VideoCapture(0)
prev_frame_time = 0
new_frame_time = 0
startTime=time.time()
handLandmarker, faceLandmarker, poseLandmarker = initModels()
while cap.isOpened():
    new_frame_time = time.time()

    ret, frame = cap.read()

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    hand_landmarker_result = handLandmarker.detect_for_video(
        mp_image,int((time.time()-startTime)*1000))
    pose_landmarket_result = poseLandmarker.detect_for_video(
        mp_image,int((time.time()-startTime)*1000))

    image = cv2.cvtColor(draw_landmarks_on_image(
        mp_image.numpy_view(), pose_landmarket_result,hand_landmarker_result), cv2.COLOR_RGB2BGR)

    # pprint(hand_landmarker_result.handedness)
    save_data(pose_landmarket_result, hand_landmarker_result, "testSave.npy")

    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(image, f"{fps:1f}", (15, 50),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
    cv2.imshow('OpenCV Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1701408139.224383  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701408139.225741  108889 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
I0000 00:00:1701408139.263250  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701408139.264393  108908 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
W0000 00:00:1701408139.264688  108640 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
I0000 00:00:1701408139.293309  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701408139.294783  108927 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5

In [ ]:
cv2.destroyAllWindows()

In [9]:
sign=['control','yes','no','thankYou','hello','iLoveYou','goodBye','please','sorry']

In [11]:
DATA_PATH = os.path.join('data')
actions = np.array(sign)
no_sequences = 100
# sequence_length = 15

In [12]:
for action in actions:
    try:
        os.mkdir(DATA_PATH+f'/{action}')
    except:
        ...

In [38]:
cap.release()
cv2.destroyAllWindows()

In [17]:
def collectData(action, length=no_sequences,start=0):
    handLandmarker, faceLandmarker, poseLandmarker = initModels()
    startTime = time.time()
    for frame_num in range(length):
        ret, frame = cap.read()

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                            data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        hand_landmarker_result = handLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))
        pose_landmarket_result = poseLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))

        image = cv2.cvtColor(draw_landmarks_on_image(
            mp_image.numpy_view(), pose_landmarket_result, hand_landmarker_result), cv2.COLOR_RGB2BGR)

        if frame_num == 0:
            cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames for {} Frame No {}'.format(action, frame_num), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            cv2.waitKey(500)
        else:
            cv2.putText(image, 'Collecting frames for {} Frame No {}'.format(action, frame_num), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

        npy_path = os.path.join(
            DATA_PATH,action, str(start+frame_num))
        save_data(pose_landmarket_result,
                    hand_landmarker_result, npy_path, )
        # Break gracefully
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

In [24]:
cap = cv2.VideoCapture(0)
# collectData('please',1,0)
# # time.sleep(5)
# collectData('please',100,100)
cap.release()
cv2.destroyAllWindows()

In [21]:
cap = cv2.VideoCapture(0)
for action in actions:
    input()
    collectData(action,100,0)
    time.sleep(5)
    collectData(action,100,100)
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1701409434.585687  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701409434.586908  113495 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
I0000 00:00:1701409434.618548  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701409434.619461  113514 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
W0000 00:00:1701409434.619696  108640 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
I0000 00:00:1701409434.642448  108640 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701409434.644405  113533 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5

In [119]:
label_map = {label: num for num, label in enumerate(actions)}
label_map

{'control': 0, 'thankyou': 1, 'iloveyou': 2, 'please': 3}

In [120]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(
                sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
np.array(sequences).shape, np.array(labels).shape

((200, 15, 225), (200,))

In [121]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int)
X.shape,Y.shape

((200, 15, 225), (200, 4))

In [122]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [123]:
X_train.shape[1:],actions.shape[0]

((15, 225), 4)

In [124]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [125]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='relu', input_shape=X_train.shape[1:]))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [127]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [128]:
model.fit(X_train, Y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
6/6 [==============================] - 8s 27ms/step - loss: 1.3787 - categorical_accuracy: 0.4167
Epoch 2/2000
6/6 [==============================] - 0s 27ms/step - loss: 1.1944 - categorical_accuracy: 0.4722
Epoch 3/2000
6/6 [==============================] - 0s 29ms/step - loss: 0.7726 - categorical_accuracy: 0.7000
Epoch 4/2000
6/6 [==============================] - 0s 31ms/step - loss: 0.6343 - categorical_accuracy: 0.8222
Epoch 5/2000
6/6 [==============================] - 0s 27ms/step - loss: 0.5382 - categorical_accuracy: 0.8556
Epoch 6/2000
6/6 [==============================] - 0s 27ms/step - loss: 0.3487 - categorical_accuracy: 0.9056
Epoch 7/2000
6/6 [==============================] - 0s 27ms/step - loss: 0.4040 - categorical_accuracy: 0.9333
Epoch 8/2000
6/6 [==============================] - 0s 29ms/step - loss: 0.2493 - categorical_accuracy: 0.9611
Epoch 9/2000
6/6 [==============================] - 0s 28ms/step - loss: 0.2080 - categorical_accuracy: 0.9444
E

KeyboardInterrupt: 

In [129]:
res = model.predict(X_test)
actions[np.argmax(res[4])], actions[np.argmax(Y_test[4])]

1/1 [==============================] - 1s 785ms/step


('iloveyou', 'iloveyou')

In [130]:
yhat = model.predict(X_test)
ytrue = np.argmax(Y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 0s 27ms/step


array([[[17,  0],
        [ 0,  3]],

       [[13,  0],
        [ 1,  6]],

       [[13,  1],
        [ 0,  6]],

       [[16,  0],
        [ 0,  4]]])

In [131]:
accuracy_score(ytrue, yhat)

0.95

In [53]:
np.expand_dims(X_test[0], axis=0)

array([[[ 0.61671913,  0.44044334, -0.6442681 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.61242396,  0.44335321, -0.55908298, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.61338311,  0.44591859, -0.5828802 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.61112583,  0.42927632, -0.32872173, ...,  0.51288551,
          0.56761646, -0.03853079],
        [ 0.61660713,  0.42383578, -0.48927504, ...,  0.49429321,
          0.57958066, -0.01583199],
        [ 0.62080556,  0.42483413, -0.71497381, ...,  0.48863906,
          0.59082407, -0.07455964]]])

In [54]:
X_test[0].shape,concatArray.shape

((15, 225), (1659,))

In [55]:
model.predict(X_test)

1/1 [==============================] - 0s 20ms/step


array([[8.4433169e-26, 9.9999994e-01, 2.3078373e-12, 1.7069017e-14],
       [9.9999994e-01, 2.1851581e-12, 1.3337194e-11, 5.1818697e-12],
       [1.6653500e-17, 5.7626761e-11, 9.9999994e-01, 1.6637562e-17],
       [9.5152330e-10, 3.8189318e-09, 5.6610895e-14, 9.9999994e-01],
       [5.8765913e-15, 4.3612984e-09, 9.9999994e-01, 1.8473335e-14],
       [9.9999994e-01, 4.4894303e-08, 2.4756556e-08, 1.2914426e-08],
       [4.1091940e-15, 9.9999994e-01, 1.9548370e-08, 5.8777673e-08],
       [7.8303637e-17, 9.9734332e-11, 9.9999994e-01, 4.3736761e-17],
       [3.5306847e-26, 9.9999994e-01, 2.2525452e-12, 4.7400214e-15],
       [6.9825840e-10, 4.1063059e-10, 1.0505851e-14, 9.9999994e-01],
       [3.8171383e-10, 5.8445926e-10, 6.8976637e-15, 9.9999994e-01],
       [2.9709948e-09, 2.8704417e-08, 3.7814939e-13, 9.9999994e-01],
       [6.2568323e-10, 1.3436660e-09, 2.0010854e-14, 9.9999994e-01],
       [4.3836480e-17, 2.3079982e-10, 9.9999994e-01, 1.0027421e-16],
       [7.4319982e-22, 9.9999994e-

In [108]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]


def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40),
                      (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return output_frame

In [132]:
try:
    # 1. New detection variables
    sequence = []
    sentence = []
    predictions = []
    threshold = 0.5
    predict= predictF = "Not init"
    cap = cv2.VideoCapture(0)
    prev_frame_time = 0
    new_frame_time = 0
    startTime = time.time()
    handLandmarker, faceLandmarker, poseLandmarker = initModels()
    while cap.isOpened():
        new_frame_time = time.time()

        ret, frame = cap.read()

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                            data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        hand_landmarker_result = handLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))
        pose_landmarket_result = poseLandmarker.detect_for_video(
            mp_image, int((time.time()-startTime)*1000))

        image = cv2.cvtColor(draw_landmarks_on_image(
            mp_image.numpy_view(), pose_landmarket_result, hand_landmarker_result), cv2.COLOR_RGB2BGR)

        # pprint(hand_landmarker_result.handedness)
        concatArray=save_data( pose_landmarket_result,
                hand_landmarker_result, npReturn=True)
        sequence.append(concatArray)
        
        if len(sequence)>15:
            predict = model.predict(np.expand_dims(sequence[-15:], axis=0))
            predictF=actions[np.argmax(predict)]
            cv2.putText(image, f"{(predict[0][0])}", (15, 410),
                        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
            cv2.putText(image, f"{predict[0][1]}", (15, 440),
                        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
            cv2.putText(image, f"{predict[0][2]}", (15, 470),
                        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)

        

        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        # cv2.putText(image, f"{fps:1f}", (15, 50),
        #             cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.putText(image, f"{predictF}", (15, 50),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
except Exception as e:
    cap.release()
    cv2.destroyAllWindows()
    raise e

I0000 00:00:1701399529.015696    4809 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701399529.017117   84330 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
I0000 00:00:1701399529.037522    4809 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701399529.038567   84349 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
W0000 00:00:1701399529.038881    4809 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
I0000 00:00:1701399529.055954    4809 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701399529.056923   84368 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5

1/1 [==============================] - 0s 23ms/step


In [133]:
model.save("model2-please")

INFO:tensorflow:Assets written to: model2-please/assets


INFO:tensorflow:Assets written to: model2-please/assets
